# Preprocesamiento de datos

In [2]:
import pandas as pd
import openpyxl 
import numpy as np
import os
from pathlib import Path
import unicodedata
from pyspark.sql.functions import desc
from pyspark.sql.functions import count, countDistinct
from pyspark.sql.functions import col

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PreprocesamientoINE").getOrCreate()

25/10/06 17:34:54 WARN Utils: Your hostname, Arielas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.100.11.238 instead (on interface en0)
25/10/06 17:34:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/06 17:34:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/06 17:34:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Funciones

In [4]:
def limpiar_nombre_columna(col, seen):
    col_clean = ''.join(
        c for c in unicodedata.normalize('NFD', str(col))
        if unicodedata.category(c) != 'Mn'
    )
    col_clean = (
        col_clean.strip()
        .lower()
        .replace(" ", "_")
        .replace(".", "_")
        .replace("-", "_")
    )

    if col_clean in seen:
        seen[col_clean] += 1
        col_clean = f"{col_clean}_{seen[col_clean]}"
    else:
        seen[col_clean] = 0
    return col_clean

def unir_excels_y_guardar_tabla(carpeta, nombre_tabla):
    folder_path = Path(carpeta)
    all_data = []

    for file in sorted(os.listdir(folder_path)):
        if file.endswith(".xlsx"):
            year = int(''.join(filter(str.isdigit, file)))
            df = pd.read_excel(folder_path / file)
            df["año"] = year
            all_data.append(df)

    df_all = pd.concat(all_data, ignore_index=True)

    seen = {}
    df_all.columns = [limpiar_nombre_columna(c, seen) for c in df_all.columns]
    df_all = df_all.loc[:, ~df_all.columns.duplicated()]

    for col in ["año", "mes_ocu", "hora_ocu", "edad_per"]:
        if col in df_all.columns:
            df_all[col] = pd.to_numeric(df_all[col], errors="coerce").astype("Int64")

    for col in df_all.columns:
        if df_all[col].dtype == "object":
            df_all[col] = df_all[col].astype(str)

    df_spark = spark.createDataFrame(df_all)
    df_spark.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(nombre_tabla)

    print(f"Guardada tabla '{nombre_tabla}' con {df_spark.count()} filas")
    df_spark.printSchema()

## Carga de datos

In [5]:

dic_fallecidos = pd.read_excel('ine/Diccionario Fallecidos y Lesionados.xlsx', header=1)
dic_hechos = pd.read_excel('ine/Diccionario Hechos.xlsx', header=1)
dic_vehiculos = pd.read_excel('ine/Diccionario Vehículos.xlsx', header= 1)

# Fallecidos
fallecidos15 = pd.read_excel('ine/Fallecidos/fallecidos2015.xlsx')
fallecidos16 = pd.read_excel('ine/Fallecidos/fallecidos2016.xlsx')
fallecidos17 = pd.read_excel('ine/Fallecidos/fallecidos2017.xlsx')
fallecidos18 = pd.read_excel('ine/Fallecidos/fallecidos2018.xlsx')
fallecidos19 = pd.read_excel('ine/Fallecidos/fallecidos2019.xlsx')
fallecidos20 = pd.read_excel('ine/Fallecidos/fallecidos2020.xlsx')
fallecidos21 = pd.read_excel('ine/Fallecidos/fallecidos2021.xlsx')
fallecidos22 = pd.read_excel('ine/Fallecidos/fallecidos2022.xlsx')
fallecidos23 = pd.read_excel('ine/Fallecidos/fallecidos2023.xlsx')

lista_fallecidos = [fallecidos15, fallecidos16, fallecidos17, fallecidos18, fallecidos19, fallecidos20, fallecidos21, fallecidos22, fallecidos23]

# Hechos
hechos15 = pd.read_excel('ine/Hechos/hechos2015.xlsx')
hechos16 = pd.read_excel('ine/Hechos/hechos2016.xlsx')
hechos17 = pd.read_excel('ine/Hechos/hechos2017.xlsx')
hechos18 = pd.read_excel('ine/Hechos/hechos2018.xlsx')
hechos19 = pd.read_excel('ine/Hechos/hechos2019.xlsx')
hechos20 = pd.read_excel('ine/Hechos/hechos2020.xlsx')
hechos21 = pd.read_excel('ine/Hechos/hechos2021.xlsx')
hechos22 = pd.read_excel('ine/Hechos/hechos2022.xlsx')
hechos23 = pd.read_excel('ine/Hechos/hechos2023.xlsx')

lista_hechos = [hechos15, hechos16, hechos17, hechos18, hechos19, hechos20, hechos21, hechos22, hechos23]

# Hechos
vehiculos15 = pd.read_excel('ine/Vehiculos/vehiculos2015.xlsx')
vehiculos16 = pd.read_excel('ine/Vehiculos/vehiculos2016.xlsx')
vehiculos17 = pd.read_excel('ine/Vehiculos/vehiculos2017.xlsx')
vehiculos18 = pd.read_excel('ine/Vehiculos/vehiculos2018.xlsx')
vehiculos19 = pd.read_excel('ine/Vehiculos/vehiculos2019.xlsx')
vehiculos20 = pd.read_excel('ine/Vehiculos/vehiculos2020.xlsx')
vehiculos21 = pd.read_excel('ine/Vehiculos/vehiculos2021.xlsx')
vehiculos22 = pd.read_excel('ine/Vehiculos/vehiculos2022.xlsx')
vehiculos23 = pd.read_excel('ine/Vehiculos/vehiculos2023.xlsx')

lista_vehiculos = [vehiculos15, vehiculos16, vehiculos17, vehiculos18, vehiculos19, vehiculos20, vehiculos21, vehiculos22, vehiculos23]


## Diccionario

In [16]:

diccionario_fallecidos = {}
variable_actual = None

for _, row in dic_fallecidos.iterrows():
    if pd.notna(row["Variable"]):  
        variable_actual = row["Variable"]
        diccionario_fallecidos[variable_actual] = {}
    
    # Intentar convertir a número
    codigo = pd.to_numeric(row["Código"], errors="coerce")
    valor = row["Valor"]

    if variable_actual and pd.notna(codigo) and pd.notna(valor):
        diccionario_fallecidos[variable_actual][int(codigo)] = str(valor)

In [17]:
diccionario_fallecidos = {var: {v: k for k, v in mapa.items()} for var, mapa in diccionario_fallecidos.items()}
diccionario_fallecidos["tipo_eve"]

{'Colisión': 1,
 'Choque': 2,
 'Vuelco': 3,
 'Caída': 4,
 'Atropello': 5,
 'Derrape': 6,
 'Embarranco': 7,
 'Encuneto': 8,
 'Ignorado': 99}

In [20]:

diccionario_hechos = {}
variable_actual = None

for _, row in dic_hechos.iterrows():
    if pd.notna(row["Variable"]):  
        variable_actual = row["Variable"]
        diccionario_hechos[variable_actual] = {}
    
    # Intentar convertir a número
    codigo = pd.to_numeric(row["Código"], errors="coerce")
    valor = row["Valor"]

    if variable_actual and pd.notna(codigo) and pd.notna(valor):
        diccionario_hechos[variable_actual][int(codigo)] = str(valor)

In [21]:
diccionario_hechos = {var: {v: k for k, v in mapa.items()} for var, mapa in diccionario_hechos.items()}
diccionario_hechos["g_hora_5"]

{'Mañana': 1, 'Tarde': 2, 'Noche': 3, 'Ignorada': 4}

In [23]:

diccionario_vehiculos = {}
variable_actual = None

for _, row in dic_vehiculos.iterrows():
    if pd.notna(row["Variable"]):  
        variable_actual = row["Variable"]
        diccionario_vehiculos[variable_actual] = {}
    
    # Intentar convertir a número
    codigo = pd.to_numeric(row["Código"], errors="coerce")
    valor = row["Valor"]

    if variable_actual and pd.notna(codigo) and pd.notna(valor):
        diccionario_vehiculos[variable_actual][int(codigo)] = str(valor)

In [24]:
diccionario_vehiculos = {var: {v: k for k, v in mapa.items()} for var, mapa in diccionario_vehiculos.items()}
diccionario_vehiculos["g_hora_5"]

{'Mañana': 1, 'Tarde': 2, 'Noche': 3, 'Ignorada': 4}

## Transformación

In [40]:
fallecidos22.replace(diccionario_fallecidos, inplace=True)
fallecidos23.replace(diccionario_fallecidos, inplace=True)

hechos22.replace(diccionario_hechos, inplace=True)
hechos23.replace(diccionario_hechos, inplace=True)

vehiculos22.replace(diccionario_vehiculos, inplace=True)
vehiculos23.replace(diccionario_vehiculos, inplace=True)

/var/folders/p9/c7lshh9n1j55x6_b26v_sx1h0000gn/T/ipykernel_12848/4218345710.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hechos22.replace(diccionario_hechos, inplace=True)
/var/folders/p9/c7lshh9n1j55x6_b26v_sx1h0000gn/T/ipykernel_12848/4218345710.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hechos23.replace(diccionario_hechos, inplace=True)
/var/folders/p9/c7lshh9n1j55x6_b26v_sx1h0000gn/T/ipykernel_12848/4218345710.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future v

In [41]:
fallecidos_junto = pd.concat(lista_fallecidos)
hechos_junto = pd.concat(lista_hechos)
vehiculos_junto = pd.concat(lista_vehiculos)

In [43]:
fallecidos_junto.to_csv('ine/fallecidos_junto.csv', index=False)
hechos_junto.to_csv('ine/hechos_junto.csv', index=False)
vehiculos_junto.to_csv('ine/vehiculos_junto.csv', index=False)